In [150]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import confusion_matrix, matthews_corrcoef, recall_score, precision_score, f1_score

import optuna

data_dir = '/home/darshana/Projects/druggable_proteins/processed_dataset'

feature_types = ['AAC', 'APAAC', 'CTD', 'DPC', 'PAAC']

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)


def evaluate_model(name, model, X_train, y_train, X_test, y_test, results_dataframe, feature_type):
    # evaluate model
    scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
    accuracy = scores.mean()

    # fit the model on the training set
    model.fit(X_train, y_train)

    # predict the test set results
    y_pred = model.predict(X_test)

    # compute the confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # calculate precision, recall (sensitivity), f1-score
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # calculate specificity
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn+fp)

    # calculate MCC
    mcc = matthews_corrcoef(y_test, y_pred)

    temp_df = pd.DataFrame({
        'feature_type': feature_type, 
        'model': name, 
        'with_hypertuning': False, 
        'accuracy': accuracy, 
        'sensitivity': recall, 
        'specificity': specificity, 
        'precision': precision, 
        'f1': f1, 
        'mcc': mcc,
        'index': f'{feature_type}_{name}_no_hypertuning'
        }, index=['index'])
    # results_dataframe is an empty dataframe to store results with the columns feature_type, model, with_hypertuning, accuracy, sensitivity, specificity, precision, f1, mcc
    return pd.concat([results_dataframe, temp_df])


def optimize_hyperparameters(name, model, objective, trials, results_dataframe, feature_type):
    def optuna_objective(trial):
        params = objective(trial)
        model_instance = model(**params)
        model_instance.fit(X_train, y_train)
        y_pred = model_instance.predict(X_test)

        # compute the confusion matrix
        cm = confusion_matrix(y_test, y_pred)

        # calculate precision, recall (sensitivity), f1-score
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        # calculate specificity
        tn, fp, fn, tp = cm.ravel()
        specificity = tn / (tn+fp)

        # calculate MCC
        mcc = matthews_corrcoef(y_test, y_pred)

        accuracy = (tp + tn) / (tp + tn + fp + fn)

        # Set user attributes
        trial.set_user_attr("precision", precision)
        trial.set_user_attr("recall", recall)
        trial.set_user_attr("f1", f1)
        trial.set_user_attr("specificity", specificity)
        trial.set_user_attr("mcc", mcc)

        return accuracy

    study = optuna.create_study(direction='maximize')
    study.optimize(optuna_objective, n_trials=trials)

    temp_df = pd.DataFrame({
        'feature_type': feature_type, 
        'model': name, 
        'with_hypertuning': True, 
        'accuracy': study.best_trial.value, 
        'sensitivity': study.best_trial.user_attrs['recall'], 
        'specificity': study.best_trial.user_attrs['specificity'], 
        'precision': study.best_trial.user_attrs['precision'], 
        'f1': study.best_trial.user_attrs['f1'], 
        'mcc': study.best_trial.user_attrs['mcc'],
        'index': f'{feature_type}_{name}_with_hypertuning'
        }, index=['index'])
    results_dataframe = pd.concat([results_dataframe, temp_df])
    return results_dataframe


# Define models
models = {
    'LogisticRegression': LogisticRegression(),
    'SVC': SVC(),
    'XGBClassifier': XGBClassifier(),
    'LGBMClassifier': LGBMClassifier()
}

models_ = {
    'LogisticRegression': LogisticRegression,
    'SVC': SVC,
    'XGBClassifier': XGBClassifier,
    'LGBMClassifier': LGBMClassifier
}

# Define objectives for hyperparameters tuning
objectives = {
    'LogisticRegression': None,  # no tuning for LogisticRegression in your original code
    'SVC': lambda trial: {
        'C': trial.suggest_float('svc_c', 1e-2, 1e2),
        'gamma': trial.suggest_float('svc_gamma', 1e-2, 1e2),
    },
    'XGBClassifier': lambda trial: {
        'learning_rate': trial.suggest_float("learning_rate", 1e-2, 0.3),
        'max_depth': trial.suggest_int("max_depth", 2, 6),
        'n_estimators': trial.suggest_int("n_estimators", 100, 1000)
    },
    'LGBMClassifier': lambda trial: {
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'max_depth': trial.suggest_int('max_depth', 2, 50),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000)
    }
}

# empty dataframe to store results with the columns feature_type, model, with_hypertuning, accuracy, sensitivity, specificity, precision, f1, mcc
results = pd.DataFrame(columns=['feature_type', 'model', 'with_hypertuning', 'accuracy', 'sensitivity', 'specificity', 'precision', 'f1', 'mcc', 'index'])

for feature_type in feature_types:

    # Load the training dataset
    train_data = pd.read_csv(f'{data_dir}/TR_{feature_type}.csv')
    test_data = pd.read_csv(f'{data_dir}/TS_{feature_type}.csv')

    # Separate features and target
    X_train = train_data.drop(columns=['label', 'id'], axis=1)
    y_train = train_data['label']

    X_test = test_data.drop(columns=['label', 'id'], axis=1)
    y_test = test_data['label']

    # Evaluate models without hyperparameters tuning
    for name, model in models.items():
        print(f"Evaluating {feature_type} {name}")
        results = evaluate_model(name, model, X_train, y_train, X_test, y_test, results, feature_type)
        print(results)

    # Optimize hyperparameters
    for name, model in models_.items():
        objective = objectives.get(name)
        if objective is not None:
            print(f"Optimizing {feature_type} {name}")
            results = optimize_hyperparameters(name, model, objective, trials=100, results_dataframe=results, feature_type=feature_type)
            print(results)

results.to_csv('results.csv', index=False)

Evaluating AAC LogisticRegression
      feature_type               model with_hypertuning  accuracy   
index          AAC  LogisticRegression            False  0.870562  \

       sensitivity  specificity  precision        f1       mcc   
index        0.875     0.924051   0.915888  0.894977  0.800787  \

                                       index  
index  AAC_LogisticRegression_no_hypertuning  
Evaluating AAC SVC
      feature_type               model with_hypertuning  accuracy   
index          AAC  LogisticRegression            False  0.870562  \
index          AAC                 SVC            False  0.895351   

       sensitivity  specificity  precision        f1       mcc   
index     0.875000     0.924051   0.915888  0.894977  0.800787  \
index     0.825893     0.940928   0.929648  0.874704  0.773776   

                                       index  
index  AAC_LogisticRegression_no_hypertuning  
index                 AAC_SVC_no_hypertuning  
Evaluating AAC XGBClassifier
    

[I 2023-05-13 10:59:23,230] A new study created in memory with name: no-name-83b75a60-d6ee-4e87-9f2b-7d7a37589a72


      feature_type               model with_hypertuning  accuracy   
index          AAC  LogisticRegression            False  0.870562  \
index          AAC                 SVC            False  0.895351   
index          AAC       XGBClassifier            False  0.878837   
index          AAC      LGBMClassifier            False  0.885530   

       sensitivity  specificity  precision        f1       mcc   
index     0.875000     0.924051   0.915888  0.894977  0.800787  \
index     0.825893     0.940928   0.929648  0.874704  0.773776   
index     0.834821     0.915612   0.903382  0.867749  0.754064   
index     0.812500     0.924051   0.910000  0.858491  0.742789   

                                       index  
index  AAC_LogisticRegression_no_hypertuning  
index                 AAC_SVC_no_hypertuning  
index       AAC_XGBClassifier_no_hypertuning  
index      AAC_LGBMClassifier_no_hypertuning  
Optimizing AAC SVC


[I 2023-05-13 10:59:23,766] Trial 0 finished with value: 0.5509761388286334 and parameters: {'svc_c': 25.24767902464712, 'svc_gamma': 1.5556320756013298}. Best is trial 0 with value: 0.5509761388286334.
[I 2023-05-13 10:59:24,265] Trial 1 finished with value: 0.5466377440347071 and parameters: {'svc_c': 70.00109073926264, 'svc_gamma': 57.93728582853489}. Best is trial 0 with value: 0.5509761388286334.
[I 2023-05-13 10:59:24,743] Trial 2 finished with value: 0.5466377440347071 and parameters: {'svc_c': 55.41118575902429, 'svc_gamma': 19.56837156908552}. Best is trial 0 with value: 0.5509761388286334.
[I 2023-05-13 10:59:25,185] Trial 3 finished with value: 0.5466377440347071 and parameters: {'svc_c': 38.71533986295983, 'svc_gamma': 37.175817923468934}. Best is trial 0 with value: 0.5509761388286334.
[I 2023-05-13 10:59:25,625] Trial 4 finished with value: 0.5466377440347071 and parameters: {'svc_c': 91.72450506146482, 'svc_gamma': 30.47001538985282}. Best is trial 0 with value: 0.550976

      feature_type               model with_hypertuning  accuracy   
index          AAC  LogisticRegression            False  0.870562  \
index          AAC                 SVC            False  0.895351   
index          AAC       XGBClassifier            False  0.878837   
index          AAC      LGBMClassifier            False  0.885530   
index          AAC                 SVC             True  0.867679   

       sensitivity  specificity  precision        f1       mcc   
index     0.875000     0.924051   0.915888  0.894977  0.800787  \
index     0.825893     0.940928   0.929648  0.874704  0.773776   
index     0.834821     0.915612   0.903382  0.867749  0.754064   
index     0.812500     0.924051   0.910000  0.858491  0.742789   
index     0.803571     0.928270   0.913706  0.855107  0.739401   

                                       index  
index  AAC_LogisticRegression_no_hypertuning  
index                 AAC_SVC_no_hypertuning  
index       AAC_XGBClassifier_no_hypertuning  


...

[I 2023-05-13 11:36:18,291] A new study created in memory with name: no-name-d4eab9ad-317c-410e-90bc-38548688c6b5


      feature_type               model with_hypertuning  accuracy   
index          AAC  LogisticRegression            False  0.870562  \
index          AAC                 SVC            False  0.895351   
index          AAC       XGBClassifier            False  0.878837   
index          AAC      LGBMClassifier            False  0.885530   
index          AAC                 SVC             True  0.867679   
index          AAC       XGBClassifier             True  0.891540   
index          AAC      LGBMClassifier             True  0.885033   
index        APAAC  LogisticRegression            False  0.874261   
index        APAAC                 SVC            False  0.870301   
index        APAAC       XGBClassifier            False  0.880982   
index        APAAC      LGBMClassifier            False  0.885727   
index        APAAC                 SVC             True  0.570499   
index        APAAC       XGBClassifier             True  0.893709   
index        APAAC      LGBMClassi

[I 2023-05-13 11:36:21,388] Trial 0 finished with value: 0.5466377440347071 and parameters: {'svc_c': 96.31857904584875, 'svc_gamma': 99.13841856406363}. Best is trial 0 with value: 0.5466377440347071.
[I 2023-05-13 11:36:24,569] Trial 1 finished with value: 0.5488069414316703 and parameters: {'svc_c': 67.93430152588688, 'svc_gamma': 6.055161297419746}. Best is trial 1 with value: 0.5488069414316703.
[I 2023-05-13 11:36:27,649] Trial 2 finished with value: 0.5466377440347071 and parameters: {'svc_c': 70.75422077826076, 'svc_gamma': 65.33240490696714}. Best is trial 1 with value: 0.5488069414316703.
[I 2023-05-13 11:36:30,819] Trial 3 finished with value: 0.5466377440347071 and parameters: {'svc_c': 15.751894888513169, 'svc_gamma': 42.24731039157137}. Best is trial 1 with value: 0.5488069414316703.
[I 2023-05-13 11:36:34,207] Trial 4 finished with value: 0.5466377440347071 and parameters: {'svc_c': 61.741736887374515, 'svc_gamma': 29.303569983714517}. Best is trial 1 with value: 0.54880

      feature_type               model with_hypertuning  accuracy   
index          AAC  LogisticRegression            False  0.870562  \
index          AAC                 SVC            False  0.895351   
index          AAC       XGBClassifier            False  0.878837   
index          AAC      LGBMClassifier            False  0.885530   
index          AAC                 SVC             True  0.867679   
index          AAC       XGBClassifier             True  0.891540   
index          AAC      LGBMClassifier             True  0.885033   
index        APAAC  LogisticRegression            False  0.874261   
index        APAAC                 SVC            False  0.870301   
index        APAAC       XGBClassifier            False  0.880982   
index        APAAC      LGBMClassifier            False  0.885727   
index        APAAC                 SVC             True  0.570499   
index        APAAC       XGBClassifier             True  0.893709   
index        APAAC      LGBMClassi

[I 2023-05-13 11:41:27,512] Trial 0 finished with value: 0.8763557483731019 and parameters: {'learning_rate': 0.117932374925602, 'max_depth': 4, 'n_estimators': 153}. Best is trial 0 with value: 0.8763557483731019.
[I 2023-05-13 11:41:30,853] Trial 1 finished with value: 0.8698481561822126 and parameters: {'learning_rate': 0.263676449979134, 'max_depth': 3, 'n_estimators': 374}. Best is trial 0 with value: 0.8763557483731019.
[I 2023-05-13 11:41:43,922] Trial 2 finished with value: 0.8763557483731019 and parameters: {'learning_rate': 0.09859988627417328, 'max_depth': 5, 'n_estimators': 890}. Best is trial 0 with value: 0.8763557483731019.
[I 2023-05-13 11:41:50,131] Trial 3 finished with value: 0.8720173535791758 and parameters: {'learning_rate': 0.2298134275342802, 'max_depth': 3, 'n_estimators': 439}. Best is trial 0 with value: 0.8763557483731019.
[I 2023-05-13 11:41:53,208] Trial 4 finished with value: 0.8763557483731019 and parameters: {'learning_rate': 0.11111080307020156, 'max_d